# Tutorial ToHyDAMOgml

6 juli 2020, Jeroen Winkelhorst, Royal HaskoningDHV

In deze notebook staat een voorbeeld uitgewerkt voor de aanmaken van een HyDAMO GML bestand. Dit voorbeeld is uitgewerkt voor drie DAMO objecten, namelijk `Hydroobject`, `DuikerSifonHevel` en `Stuw`. De benodigde bronbestanden zijn bijgevoegd in de map `examples/gdb`.

Meer informatie over het HyDAMO datamodel vind je hier: http://www.nhi.nu/nl/index.php/uitvoering/module-oppervlaktewater/ 


Ben je niet bekend met Jupyter Notebooks? Lees dan deze korte introductie (engels): https://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb

De installatie van de ToHyDAMOgml tool staat toegelicht in de Readme van deze package. 

### Inladen functies

Als eerste wordt de map waarin de `hydamo_gml` package staat toegevoegd aan het `systeem path`. Vervolgens worden de benodigde dependencies ingeladen. 

In [ ]:
import sys
# Tussen de r'' staat de verwijzing naar de map van `hydamo_gml`. Voor dit voorbeeld staat de relatieve verwijzing goed ingesteld.
sys.path.append(r'../..')

# Deze map 'hydamo_gml' map bevat tenminste de volgende submappen:
# - `examples\`
# - `hydamo_gml\`
# - `scripts\`
# - `src\`

# Importeer de benodigde packages
import os
from tohydamogml.hydamo_table import HydamoObject

%matplotlib notebook

### Hydro object
Als eerste gaan we het object Hydro object (watergangen) omzetten naar een GML bestand. Daarvoor doorlopen we de volgende stappen:
1. Inladen configuratiebestand hydroobject.json
2. Aanmaken Python HydamoObject. In dit object is de GML code opgebouwd.
3. Uitvoeren validatie van de GML code
4. wegschrijven van de GML code naar een GML bestand

#### Configuratie bestanden
In de configuratie bestanden staat per HyDAMO object gedefinieerd hoe het object is opgebouwd. De configuratie is vastgelegd in JSON bestanden die een vergelijkbare opbouw hebben als Python dictionaries. De opbouw van de JSON bestanden is toegelicht in een notitie die te vinden is in de map `docs`.

In dit voorbeeld gaan we als eerste de watergangen omzetten. Daarvoor hebben we het json bestand van `HydroObject` nodig. 

In [ ]:
json_hydroobject = r"json/hydroobject.json"

Neem eens een kijkje in dit json bestand. Dat bestand kun je openen met een text editor zoals `notepad`. In dit bestand staat alle informatie die nodig is een HyDAMO object aan te maken. Het is belangrijk dat de verwijzing naar de brondatabase (beheerregister) goed staat in het JSON bestand. 

#### Aanmaken HyDAMO object
Voor dit voorbeeld staan de brongegevens goed, dus we laden het hydroobject in. Hiervoor is een internetverbinding noodzakelijk omdat gml protocollen van internet worden gehaald. 

Het is mogelijk om het via het attribuut `mask` een polygon mee te geven om de uitvoer te beperken tot een bepaald gebied. Dit is optioneel.

(het is mogelijk dat je een foutmelding krijgt over een 'data rate limit'. Restart dan deze notebook van de python prompt met het volgende commando: `jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10`)

In [ ]:
gebied = r"shp/oosterwolde_clip.shp"
obj = HydamoObject(json_hydroobject, mask=gebied)

In [ ]:
%debug

De GML output is na het aanmaken weergegeven maar nog niet weggeschreven naar een bestand. 

De informatie is in tabelform in te zien via het commanda `obj.gdf`. 

In [ ]:
obj.gdf

De geometrie kan eenvoudig geplot worden met het commando `obj.gdf.plot()` 

In [ ]:
obj.gdf.plot()

#### Validatie HyDAMO object
Het is mogelijk om de gml output te valideren met behulp van bijgeleverde XSD bestanden. Het is daarbij mogelijk om de validatie fouten weg te schrijven naar een shape.

In [ ]:
obj.validate_gml(write_error_log=True)

Als de validatie fouten bevat zijn deze weggeschreven naar een logbestand in de map `wvv_notebook/log`. De log bestaat uit een shape of csv bestand. Zoek dit bestand eens op en bekijk de inhoud.

Het XSD bestand dat wordt gebruikt voor validatie staat in de map `src/xsd`. Als er een nieuwere versie beschikbaar is op `https://github.com/erikderooij/nhi/tree/master/schema` kunnen de bestanden in deze map worden overschreven.

#### Exporteren gml naar bestand
Na de validatie exporteren we de gml naar een bestand. De `write_gml` functie voert automatisch een validatie uit. Als je de validatie al hebt uitgevoerd (zoals we net gedaan hebben in de vorige stap) kun je de validatie overslaan met de optie `skip_validation=True`. Als er validatie errors in het het object zitten wordt is de standaard instelling dat er geen output wordt weggeschreven. Wil je dit toch, gebruik dan de optie `ignore_errors=True`.

We schrijven het gml bestand weg naar de map 'output' en negeren de validatie errors:

In [ ]:
obj.write_gml(r"output", ignore_errors=True, skip_validation=True)

### Object stuw
Het object stuw verschilt van HydroObject omdat bij dit object in de JSON configuratie wordt verwezen naar attribuutspecifieke functies. De locatie van de functies moet in aanvulling op de json locatie worden ingeladen. In het volgende voorbeeld wordt het object stuw weggeschreven naar een gml bestand.

In [ ]:
json_stuw = r"json/stuw.json"
attr_function = r"json/attribute_functions.py"
obj_stuw = HydamoObject(json_stuw, file_attribute_functions=attr_function)
obj_stuw.write_gml(r"output", ignore_errors=True, skip_validation=False)

### Opdrachten

In deze opdracht ga je als eerste het DuikerSifonHevel object exporteren naar GML. Vervolgens ga je de JSON aanpassen zodat je een extra kolom exporteert.

1. Open de JSON van het DuikerSifonHevel object (`examples\wvv_notebook\json\duikersifonhevel.json`) en probeer de opbouw te begrijpen. De documentatie in de map `docs` kan hierbij helpen.
2. Open de gis laag `duikersifonhevel` uit het beheerregister (`examples\wvv_notebook\gdb\Breg_Hydamo.gdb`) en bekijk hoe de JSON verwijst naar de kolommen in het beheerregister
3. Laad het object DuikerSifonHevel in voor het gebied oosterwolde ( `shp/clip_oosterwolde.shp`) en plot de duikers. De aanpak hiervoor is vergelijkbaar met het voorbeeld van het object Stuw

4. De output van de duikers is gefilterd. Op welk attribuut is het object gefilterd en wat betekent dit voor de resultaten? De filtering staat gedefinieerd in het json bestand.
<br>

Hint:
- 1	planvorming
- 2	realisatie
- 3	gerealiseerd
- 4	buiten bedrijf
- 5	niet meer aanwezig
- 7	te verwijderen
- 99	onbekend



5. Maak een kopie van het JSON bestand van duikersifonhevel en filter hierin de duikers op 'niet meer aanwezig' en 'te verwijderen'. Plot het resultaat. 

6. Achterhaal hoe de kolom ruwheidswaarde in het JSON bestand wordt afgeleid. 
    - Wat is de kolom uit het beheerregister waar naar wordt verwezen? 
    - Welk type output zie je terug in de GML? 
    - Hoe is de data omgezet?

Hint: kijk in attribute_functions.py

7. We gaan nu een extra kolom aan de gml van het duikersifonhevel object toevoegen, namelijk de kolom kolom `opmerking` (`OPMERKING`). 
    - Maak een kopie van het originele JSON bestand
    - voeg de extra kolom `opmerking` (`OPMERKING`) toe
    - kies als default waarde "Geen opmerking" en "required": false
    - maak een hydamo object aan en bekijk de tabeloutput
    - zie je de default waarde terug? pas de JSON aan en zet "required": true. Wat zie je nu in de tabeloutput?

hint: Het type is "String"

8. Exporteer het aangepaste duikersifonhevel object naar gml

<b>Dit is het einde van de tutorial. Je hebt nu de meest belangrijke stappen doorlopen om te begrijpen hoe de tool functioneert en hoe je informatie aanpast. Succes!</b>